In [1]:
from flask import Flask, request, redirect, url_for
from pymongo import MongoClient
import subprocess

In [2]:
app = Flask(__name__)

def render_template(template_name, data):
    ejsRenderer = subprocess.Popen(
        ["ejs", f"templates/{template_name}"], stdin=subprocess.PIPE, stdout=subprocess.PIPE)
    rendered = ejsRenderer.communicate(bytes(json.dumps(data), "utf8"))[0]
    ejsRenderer.stdin.close()
    ejsRenderer.wait()
    return rendered

In [3]:
client = MongoClient('localhost', 27017)
db = client["flashcard_db"]
print("DB ----> ",db)
flashcards_collection = db["flashcards"]
print("FLASHCARD COLLECTION ----> ",flashcards_collection)

DB ---->  Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'flashcard_db')
FLASHCARD COLLECTION ---->  Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'flashcard_db'), 'flashcards')


In [4]:
@app.route('/')
def index():
    flashcards = list(flashcards_collection.find())
    print("Flash Cards",flashcards)
    return render_template('index.ejs', flashcards=flashcards)

@app.route('/add_flashcard', methods=['GET', 'POST'])
def add_flashcard():
    if request.method == 'POST':
        question = request.form['question']
        answer = request.form['answer']
        flashcards_collection.insert_one({'question': question, 'answer': answer})
        return redirect(url_for('index'))
    return render_template('add_flashcard.ejs')

@app.route('/quiz')
def quiz():
    flashcards = list(flashcards_collection.find())
    return render_template('quiz.ejs', flashcards=flashcards)

if __name__ == '__main__':
    app.run(debug=True)


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with stat
0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.
Traceback (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/Users/debanilguha/opt/anaconda3/envs/deb/lib/python3.11/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/Users/debanilguha/opt/anaconda3/envs/deb/lib/python3.11/site-packages/traitlets/config/application.py", line 1074, in launch_instance
    app.initialize(argv)
  File "/Users/debanilguha/opt/anaconda3/envs/deb/lib/python3.11/site-packages/traitlets/config/application.py", line 118, in inner
    return method(a

SystemExit: 1

/Users/debanilguha/opt/anaconda3/envs/deb/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
